# 05c: Calculate combustion impacts for hydrogen and gases

Equivalently as for liquid fuels, we also calculate the impacts from combustion of hydrogen and gases to later add to the production impacts.

In [1]:
%run common_definitions.py

In [2]:
import pandas as pd
import numpy as np
import xarray as xr

import brightway2 as bw

In [8]:
bw.projects.set_current(BW_PROJECTNAME)
output_fp = "../output/" + BW_PROJECTNAME

### Get lookup dictionary for database codes of biosphere flows

In [9]:
def category_tuple_to_str(tpl):
    return "::".join(list(tpl))

In [19]:
DF_BASE = pd.read_csv("../data/hydrogen_and_gases_combustion_efs.csv")
BS = bw.Database("biosphere3")

flow2code = DF_BASE.groupby(["name", "categories"]).count().reset_index()[["name", "categories"]]
matches = []
for name, cat in zip(flow2code["name"], flow2code["categories"]):
        matches_name = [flow for flow in BS if flow["name"] == name]
        matches_cat = [flow for flow in matches_name if category_tuple_to_str(flow["categories"]) == cat]
        if len(matches_cat) == 1:
            matches.append(matches_cat[0])
        else:
            raise ValueError("Not exactly one matching flow found for {}, {}!".format(name, cat))

flow2code["code"] = [flow["code"] for flow in matches]

In [20]:
flow2code = flow2code.set_index(["name", "categories"])["code"]

### Create full dataframe of emission factors from base fuels

In [21]:
def get_biosphere_exchanges(base_fuel):
    df = DF_BASE[DF_BASE["fuel name"] == base_fuel][["name", "categories", "amount"]]

    # get codes from lookup
    df["code"] = [flow2code.loc[(n, c)] for n, c in zip(df["name"], df["categories"])]

    return df

In [8]:

def get_suffix(row):
    return ", burned in " + row["base_fuel"].split(", ")[-1]

def get_ncv(row):
    prod = row["reference product"]

    df_ncvs = pd.read_csv("../data/NCVs_v2.csv")
    return df_ncvs.set_index("product name").loc[prod]["NCV in MJ/product"]

In [22]:
mapping = pd.read_csv("../mappings/hydrogen_and_gases_2_combustion_exchanges.csv")
long2short = dict(zip(list(mapping["ecoinvent name"]), list(mapping["short name"])))

In [23]:
long2short

{'petroleum and gas production, onshore': 'natural gas, onshore',
 'petroleum and gas production, offshore': 'natural gas, offshore',
 'biomethane production, from biogas upgrading, using amine scrubbing': 'biomethane, amine scrubbing',
 'treatment of biowaste by anaerobic digestion': 'biogas, biowaste',
 'treatment of sewage sludge by anaerobic digestion': 'biogas, sewage sludge',
 'treatment of used vegetable cooking oil by anaerobic digestion': 'biogas, used vegetable cooking oil',
 'hydrogen production, gaseous, 100 bar, from methane pyrolysis': 'hydrogen, methane pyrolysis',
 'hydrogen production, gaseous, 25 bar, from gasification of woody biomass in entrained flow gasifier, at gasification plant': 'hydrogen, woody biomass',
 'hydrogen production, gaseous, 25 bar, from gasification of woody biomass in entrained flow gasifier, with CCS, at gasification plant': 'hydrogen, woody biomass, CCS',
 'hydrogen production, gaseous, 25 bar, from thermochemical water splitting, at solar towe

In [25]:
dflist = []
for idx, row in mapping.iterrows():
    df_bs = get_biosphere_exchanges(row["base_fuel"])
    df_bs["ecoinvent name"] = row["ecoinvent name"]
    dflist.append(df_bs)

all_efs = pd.concat(dflist, axis=0)
all_efs.to_csv("../data/hydrogen_and_gases_combustion_all_efs.csv", index=False)

### Calculate the impacts for the needed LCIA methods

In [26]:
def get_cfs(keys, methods):
    # initialize values
    cfs = np.zeros((len(keys), len(methods)))

    # loop through methods
    for j, m in enumerate(methods):
        print("Collecting CFs for method {} of {}, {}".format(j+1, len(methods), str(m)))
        for flow in bw.Method(m).load():
            key = flow[0][1]
            if key in keys:
                i = keys.index(key)
                cfs[i,j] = flow[1]

    # construct array
    da = xr.DataArray(
        cfs,
        coords={
            "code": keys,
            "LCIA method": [", ".join(list(m)) for m in methods]
        }
    )

    return da

In [27]:
df2 = pd.read_csv("../data/mfs_all_factors.csv").dropna(subset="LCIA method")

needed_methods = list(df2["LCIA method"].unique())
methods = [m for m in list(bw.methods) if ", ".join(list(m)) in needed_methods]

cc_methods = [
    ("IPCC 2021", "climate change: biogenic", "GWP 100a"),
    ("IPCC 2021", "climate change: land use", "GWP 100a"),
    ("IPCC 2021", "climate change: fossil", "GWP 100a")
]
methods = methods + cc_methods

print("{} methods needed, {} methods found".format(len(needed_methods)+len(cc_methods), len(methods)))

73 methods needed, 73 methods found


In [28]:
unique_keys = list(all_efs["code"].unique())

In [29]:
cfs = get_cfs(unique_keys, methods)

### Build emission factor data array

In [30]:
ef_xr = all_efs.set_index(["ecoinvent name", "code"])["amount"].to_xarray().fillna(0)

### Calculate impacts

Impacts are simply the (matrix) product of characterization and emission factors.

In [31]:
impacts = (ef_xr * cfs).sum(dim="code")
impact_df = impacts.to_pandas()

In [32]:
impact_df.to_csv(output_fp+"/impacts_hydrogen_and_gases_combustion.csv")